In [ ]:
# ----------------------------------------
# Constantes para el juego Hundir la flota
# ----------------------------------------

import numpy as np

# Dimensión del tablero
dim_tablero = 10

# definimos los barcos
barcos = {
    "barco1": (1, 4), # siendo el primer número la eslora y el segundo número la cantidad de barcos
    "barco2": (2, 3),
    "barco3": (3, 2),
    "barco4": (4, 1)
}

# definimos los símbolos para el tablero
agua = "~"
barco = "B"
impacto = "X"
fallo = "O"
desconocido = "." # para el tablero del enemigo donde aun no se ha disparado

# IDs jugadores ## Añadido por ANA
jugador = "Jugador"
maquina = "Maquina"

# damos una semilla para la generación aleatoria
np.random.seed(42)


In [ ]:
# --------------------------------------------
#  Clase Tablero para el juego Hundir la flota
# --------------------------------------------

import numpy as np
import random

class Tablero:
    def __init__(self, id_jugador: str):
        self.id_jugador = id_jugador
        self.dimension = dim_tablero
        self.tablero_barcos = np.full((dim_tablero, dim_tablero), agua) # con los barcos propios
        self.tablero_disparos = np.full((dim_tablero, dim_tablero), desconocido) # con los disparos hechos al enemigo
        self.vidas = 0  # total de casillas con barco


#### Codigo ajustado por ANA (con el codigo de antes los barcos solo se pueden poner de manera horizontal. se pueden salir del tablero o se pueden pisar entre ellos)

# 1. Función para generar las coordenadas de los barcos,
# dado una lista de barcos que contenga dos parámetros, 
# el primero la eslora, y el segundo la cantidad de barcos iguales.
# Se tiene en cuenta que no se salga del tablero y que no pise otro barco

    
    def colocar_barcos_aleatorio(self, barcos, dim_tablero):

        lista_coord_ocupadas = []  # En esta lista se guardan todas las casillas ocupadas

        # recorremos todos los barcos que queremos colocar
        for (eslora, cantidad) in barcos.values():
            for i in range(cantidad):

                # se elige la orientacion del barco al azar (True: barco horizontal y False: barco vertical)
                es_horizontal = random.choice([True, False])  # random.choice elige al azar True o False
            
                # Intenta colocar el barco (máximo 100 intentos)
                colocado = False  # todavía no hemos colocado el barco
                intentos = 0  # contador de intentos
            
                while not colocado and intentos < 100:  #mientras el barco no esté colocado y no hayamos llegado a los 100 intentos, sigue intentando buscar un sitio para el barco
                
                    if es_horizontal: # Si el barco esta orientado horizontalmente 
                        x = np.random.randint(0, dim_tablero - 1)  # elige una fila al azar entre 0 y la ultima fila (9)
                        y = np.random.randint(0, dim_tablero - eslora)  # Columna donde empieza el barco, asegurándonos de que cabe hacia la derecha

                        # generar coordenadas del barco
                        coords = [] #crea una lista donde guardaremos todas las coordenadas que ocupa el barco.
                        for j in range(eslora):
                            coords.append([x, y + j])

                    # Si el barco esta orientado verticalmente (barco va hacia abajo)
                    else:
                        x = np.random.randint(0, dim_tablero - eslora)  # la fila donde empieza el barco, asegurándonos de que cabe hacia abajo
                        y = np.random.randint(0, dim_tablero - 1)  # Elige cualquier columna (0-9)

                        # generar coordenadas del barco
                        coords = [] #crea una lista donde guardaremos todas las coordenadas que ocupa el barco.
                        for j in range(eslora):
                            coords.append([x + j, y]) 

                    # Comprobar si todas las casillas están libres. Queremos saber si el barco que estamos intentando colocar pisa otro barco. 
                    todas_libres = True # empezamos asumiendo que el barco nuevo no pisa otro barco
                    for c in coords: #miramos cada casilla del barco actual una por una
                        if c in lista_coord_ocupadas: #aqui miramos si las casillas ya estan ocupadas por otro barco
                            todas_libres = False  # si hay una casilla ocupada marcamos que no está libre con el False
                            break  # salimos del bucle, no hace falta seguir comprobando

                    # Si todas las casillas del barco están libres, entonces podemos colocar el barco
                    if todas_libres:
                        for c in coords: #coords contiene las coordenadas del barco
                            lista_coord_ocupadas.append(c)  # añadimos cada casilla del barco a la lista lista_coord_ocupadas, tiene todas las coordenadas de todos los barcos del tablero
                        colocado = True  # Ya está colocado, salir del while
                        # comento el print para que deje de salir por pantalla ahora que sabemos que funciona
                        # print(f"El barco con eslora {eslora} está colocado en: {coords}")

                    intentos += 1  # incrementa contador

                # si después de 100 intentos no se pudo colocar el barco, avisar
                if not colocado:
                    print(f"No se pudo colocar el barco de eslora {eslora}")

        return lista_coord_ocupadas
    
    
    # Colocar barcos SIN SALIRSE y SIN SOLAPARSE # Cris
    # ------------------------------------------------------------
    def colocar_barcos(self):
        coords_ocupadas = []

        for (eslora, cantidad) in variables.BARCOS.values():
            for _ in range(cantidad):

                colocado = False
                while not colocado:

                    # orientación aleatoria
                    orientacion = np.random.choice(["H", "V"])

                    if orientacion == "H":
                        x = np.random.randint(0, self.dimension)
                        y = np.random.randint(0, self.dimension - eslora + 1)

                        nuevas = [(x, y + i) for i in range(eslora)]

                    else:  # Vertical
                        x = np.random.randint(0, self.dimension - eslora + 1)
                        y = np.random.randint(0, self.dimension)

                        nuevas = [(x + i, y) for i in range(eslora)]

                    # Comprobar solapamientos
                    if any(coord in coords_ocupadas for coord in nuevas):
                        continue
                    else:
                        coords_ocupadas.extend(nuevas)
                        colocado = True

        # Marcar barcos en tablero
        for (x, y) in coords_ocupadas:
            self.tablero_barcos[x, y] = variables.BARCO
            self.vidas += 1

    # ------------------------------------------------------------
    # Mostrar tablero de barcos
    # ------------------------------------------------------------
    def mostrar_barcos(self):
        for fila in self.tablero_barcos:
            print(" ".join(fila))
        return

In [11]:
# --------------------------------------------
#  Funciones para el juego Hundir la flota
# --------------------------------------------

# Esta función coloca los barcos en el tablero, 
# dada la lista de coordenadas ocupadas
# y el tablero original
def colocar_barcos(tablero, lista_coord_ocupadas):
    for [x, y] in lista_coord_ocupadas:
       tablero[x, y] = "B"
    return tablero

# Función para disparar el jugador
def disparo_jugador(tablero, x, y):
    if tablero[x, y] == barco:
        tablero[x, y] = impacto
        print("Tocado")
    elif tablero[x, y] == agua:
        tablero[x, y] = fallo
        print("Agua")
    else:
        print("Ya has disparado aquí")
    return tablero


# Función para disparar de forma aleatoria la máquina
def disparo_maquina(tablero):
    # esta función toma valores aleatorios de x e y
    x = np.random.randint(0,9)
    y = np.random.randint(0,9)
    if tablero[x, y] == barco:
        tablero[x, y] = impacto
        print("Tocado")
    elif tablero[x, y] == agua:
        tablero[x, y] = fallo
        print("Agua")
    else:
        print("Ya has disparado aquí")
    return tablero

# Función para mostrar el tablero de la máquina sin desvelar dónde están los barcos
def mostrar_tablero_maquina_oculto(tablero):
    tablero_oculto = tablero.copy()
    tablero_oculto[tablero_oculto == barco] = desconocido
    tablero_oculto[tablero_oculto == agua] = desconocido
    return tablero_oculto

In [ ]:
# --------------------------------------------
#  A partir de aquí sería el main.py
# --------------------------------------------

# Hago las llamadas a los otros archivos
from variables import *
from tablero import *
from funciones import *

# Comprobación de que se está ejecutando el main
print("Que comience el juego...")

# Genero los tableros de jugador y máquina
# 1. gereno tablero vacío
t = Tablero("Jugador")
tablero_jugador = t.tablero_barcos
t_m = Tablero("Máquina")
tablero_maquina = t_m.tablero_barcos

# 2. genero la lista de coordenadas de barcos
lista_coord_barcos_jugador = t.colocar_barcos_aleatorio(barcos, dim_tablero)
lista_coord_barcos_maquina = t_m.colocar_barcos_aleatorio(barcos, dim_tablero)

# 3. coloco los barcos en el tablero
tablero_barcos_jugador = colocar_barcos(tablero_jugador, lista_coord_barcos_jugador)
tablero_barcos_maquina = colocar_barcos(tablero_maquina, lista_coord_barcos_maquina)

# 4. muestro los tableros. para el jugador muestro el tablero tal cual
# para la máquina, primero creo el tablero oculto y luego lo muestro
print("Tablero jugador \n")
print(tablero_barcos_jugador)

tablero_barcos_maquina_oculto = mostrar_tablero_maquina_oculto(tablero_barcos_maquina)
print("\n Tablero máquina \n")
print(tablero_barcos_maquina_oculto)

Tablero jugador 

[['~' '~' '~' 'B' '~' '~' '~' 'B' 'B' '~']
 ['~' 'B' 'B' 'B' '~' 'B' '~' 'B' 'B' '~']
 ['~' '~' '~' 'B' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' 'B' '~' '~' '~' '~' '~' '~']
 ['~' '~' 'B' 'B' '~' '~' 'B' 'B' 'B' '~']
 ['~' 'B' '~' 'B' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' 'B' '~' '~' '~' 'B' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']]

 Tablero máquina 

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']]


In [ ]:
# Aquí habría que comenzar con un while por turnos para ir disparando y, después de cada turno, 
# mostrar los tableros del jugador normal, y el tablero de la máquina que primero tendría que pasar
# por la función mostrar_tablero_maquina_oculto


# PARTE MÁS IMPORTANTE QUE FALTA



# Disparo del jugador y mostramos tablero de la máquina oculto
tablero_barcos_maquina = disparo_jugador(tablero_barcos_maquina, 0, 4)
tablero_barcos_maquina_oculto = mostrar_tablero_maquina_oculto(tablero_barcos_maquina)
print("\n Tablero máquina \n")
print(tablero_barcos_maquina_oculto)


Agua

 Tablero máquina 

[['.' '.' '.' '.' 'O' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']]


In [21]:
# Disparo de la máquina y mostramos tablero del jugador
# en este caso no hay que poner coordenadas porque se eligen de forma aleatoria
tablero_barcos_jugador = disparo_maquina(tablero_barcos_jugador) 
print("Tablero jugador \n")
print(tablero_barcos_jugador)

Agua
Tablero jugador 

[['~' '~' '~' 'B' '~' '~' '~' 'B' 'B' '~']
 ['~' 'B' 'B' 'B' '~' 'B' '~' 'B' 'B' '~']
 ['O' '~' '~' 'B' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' 'B' '~' '~' '~' '~' '~' '~']
 ['~' '~' 'B' 'B' '~' '~' 'B' 'B' 'B' '~']
 ['~' 'B' '~' 'B' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' 'B' '~' '~' '~' 'B' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']
 ['~' '~' '~' '~' '~' '~' '~' '~' '~' '~']]
